In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv
load_dotenv()
encoder = SentenceTransformer('all-MiniLM-L6-v2') 

qdrant = QdrantClient(
    os.getenv("QDRANT_ENDPOINT"), 
    prefer_grpc=True,
    api_key=os.getenv("QDRANT_API_KEY")
)

/opt/homebrew/Caskroom/miniconda/base/envs/llms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
documents = [
{ "name": "The Time Machine", "description": "A man travels through time and witnesses the evolution of humanity.", "author": "H.G. Wells", "year": 1895 },
{ "name": "Ender's Game", "description": "A young boy is trained to become a military leader in a war against an alien race.", "author": "Orson Scott Card", "year": 1985 },
{ "name": "Brave New World", "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.", "author": "Aldous Huxley", "year": 1932 },
{ "name": "The Hitchhiker's Guide to the Galaxy", "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.", "author": "Douglas Adams", "year": 1979 },
{ "name": "Dune", "description": "A desert planet is the site of political intrigue and power struggles.", "author": "Frank Herbert", "year": 1965 },
{ "name": "Foundation", "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.", "author": "Isaac Asimov", "year": 1951 },
{ "name": "Snow Crash", "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.", "author": "Neal Stephenson", "year": 1992 },
{ "name": "Neuromancer", "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.", "author": "William Gibson", "year": 1984 },
{ "name": "The War of the Worlds", "description": "A Martian invasion of Earth throws humanity into chaos.", "author": "H.G. Wells", "year": 1898 },
{ "name": "The Hunger Games", "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.", "author": "Suzanne Collins", "year": 2008 },
{ "name": "The Andromeda Strain", "description": "A deadly virus from outer space threatens to wipe out humanity.", "author": "Michael Crichton", "year": 1969 },
{ "name": "The Left Hand of Darkness", "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.", "author": "Ursula K. Le Guin", "year": 1969 },
{ "name": "The Three-Body Problem", "description": "Humans encounter an alien civilization that lives in a dying system.", "author": "Liu Cixin", "year": 2008 }
]

In [ ]:
qdrant.recreate_collection(
	collection_name="my_books",
	vectors_config=models.VectorParams(
		size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
		distance=models.Distance.COSINE
	)
)

In [7]:
list(qdrant.get_collection(collection_name="my_books"))

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', 13),
 ('indexed_vectors_count', 0),
 ('points_count', 13),
 ('segments_count', 2),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None)),
 ('payload_schema', {})]

In [4]:
qdrant.upload_records(
	collection_name="my_books",
	records=[
		models.Record(
			id=idx,
			vector=encoder.encode(doc["description"]).tolist(),
			payload=doc
		) for idx, doc in enumerate(documents)
	]
)

In [5]:
hits = qdrant.search(
	collection_name="my_books",
	query_vector=encoder.encode("alien invasion").tolist(),
	limit=3
)
for hit in hits:
	print(hit.payload, "score:", hit.score)

{'year': 1898, 'description': 'A Martian invasion of Earth throws humanity into chaos.', 'author': 'H.G. Wells', 'name': 'The War of the Worlds'} score: 0.5700932741165161
{'description': 'A comedic science fiction series following the misadventures of an unwitting human and his alien friend.', 'year': 1979, 'author': 'Douglas Adams', 'name': "The Hitchhiker's Guide to the Galaxy"} score: 0.5040467977523804
{'description': 'Humans encounter an alien civilization that lives in a dying system.', 'year': 2008, 'author': 'Liu Cixin', 'name': 'The Three-Body Problem'} score: 0.45902931690216064
